In [3]:
from datasets import load_dataset

In [4]:
train_dataset = load_dataset("sentence-transformers/all-nli", "pair-score", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/69.5M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/942069 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/19657 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19656 [00:00<?, ? examples/s]

In [5]:
train_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 942069
})

In [6]:
train_dataset[0]

{'sentence1': 'A person on a horse jumps over a broken down airplane.',
 'sentence2': 'A person is training his horse for a competition.',
 'score': 0.5}

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(train_dataset)
df.head()

,sentence1,sentence2,score
0,A person on a horse jumps over a broken down airplane.,A person is training his horse for a competition.,0.5
1,A person on a horse jumps over a broken down airplane.,"A person is at a diner, ordering an omelette.",0.0
2,A person on a horse jumps over a broken down airplane.,"A person is outdoors, on a horse.",1.0
3,Children smiling and waving at camera,They are smiling at their parents,0.5
4,Children smiling and waving at camera,There are children present,1.0


In [8]:
df["score"].unique()

array([0.5, 0. , 1. ])

In [9]:
df.sample(20, random_state=42)

,sentence1,sentence2,score
762721,you had to do some work and,you had to do a lot of work for the army.,0.5
674347,The others were grouped together at a little distance away.,There was a group of people at a short distance.,1.0
2362,A small child in blue shirt and blue jeans is running through a wooded path over dry leaves.,A child is running over leaves.,1.0
913903,"Wood Quay, on the south bank of the river (downhill from the arch and dominated by the featureless offices of the Dublin Corporation), is the site of the original Viking settlement and its clever recreation, The Viking Adventure (Tuesday Saturday 10am 4:30pm, closed Sunday and Monday; adults IRa4.","The original Viking settlement and its modern-day recreation are located on Wood Quay, downhill from the arch.",1.0
204130,An older man smiling and holding twine.,an older man is going to tie a knot,0.5
220223,A group of people in life jackets standing on a large boulder in the mountains.,A group of people are about to go swimming.,0.5
277414,Two men enjoying a beer together.,The two men are drunk.,0.5
788967,"The appeals court rapped the agency for its scare tactics, saying it must base its conclusions on solid facts and a realistic appraisal of the danger rather than on vague fears extrapolated beyond any foreseeable threat.",The court supported the agency and thought they way they dealt with things was friendly.,0.0
722422,Texas yeah i'm in California right now i'm i'm originally from North Carolina though i,I love California more than I love North Carolina.,0.5
791949,The clothes didn't fit.,The dress didn't fit.,0.5


In [10]:
from sentence_transformers import SentenceTransformer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
model_name = "answerdotai/ModernBERT-base"
model = SentenceTransformer(model_name, device="cuda")

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [12]:
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: ModernBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [13]:
from sentence_transformers.losses import CoSENTLoss
loss = CoSENTLoss(model)

In [24]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/ModernSBERT",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,
    bf16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    report_to=None,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# Load the STSB dataset (https://huggingface.co/datasets/sentence-transformers/stsb)
eval_dataset = load_dataset("sentence-transformers/all-nli", "pair-score", split="dev")

# Initialize the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="all-nli-dev",
)

In [17]:
test_dataset = load_dataset("sentence-transformers/all-nli", "pair-score", split="test")

In [18]:
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
)

In [19]:
test_evaluator(model)

W0320 10:38:11.211000 3395 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


{'pearson_cosine': np.float64(0.1370168450138532),
 'spearman_cosine': np.float64(0.1631801108514827)}

In [25]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

In [26]:
model.save_pretrained("ModernSBERT-base-all-nli/final")

In [27]:
model = SentenceTransformer("./ModernSBERT-base-all-nli/final")

In [28]:
test_evaluator(model)

{'pearson_cosine': np.float64(0.15070938488904378),
 'spearman_cosine': np.float64(0.1762633751467881)}

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
